In [1]:
import sys
sys.path.insert(1, '/home/b3arjuden/crocket')

In [18]:
from concurrent.futures import as_completed
from json import load as json_load
from requests.exceptions import ConnectionError
from requests_futures.sessions import FuturesSession
from requests.exceptions import ConnectTimeout, ConnectionError, ProxyError, ReadTimeout

from crocket.bittrex.bittrex2 import Bittrex, format_bittrex_entry, return_request_input

In [3]:
PROXY_LIST_PATH = '/home/b3arjuden/crocket/proxy_list.txt'
BITTREX_CREDENTIALS_PATH = '/home/b3arjuden/bittrex_credentials.json'

In [4]:
markets = ['BTC-LTC'] * 200

with open(PROXY_LIST_PATH, 'r') as f:
    proxies = f.read().splitlines()
    
with open(BITTREX_CREDENTIALS_PATH, 'r') as f:
    BITTREX_CREDENTIALS = json_load(f)

In [5]:
bittrex = Bittrex(api_key=BITTREX_CREDENTIALS.get('key'),
                  api_secret=BITTREX_CREDENTIALS.get('secret'),
                  dispatch=return_request_input,
                  api_version='v1.1')

In [6]:
def configure_ip(ip):
    
    return {
        'http': ip,
        'https': ip
    }

def process_response(session, response):
    
    try:
        response.data = response.json()
    except:
        response.data = {
               'success': False,
               'message': 'NO_API_RESPONSE',
               'result': None
            }

In [19]:
futures = []
response_dict = {}
num_proxies = len(proxies)

with FuturesSession(max_workers=20) as session:

    for index in range(len(markets)):
        market = markets[index]
        request_input = bittrex.get_market_history(market)

        proxy = configure_ip(proxies[index])
        url = request_input.get('url')
        headers = {"apisign": request_input.get('apisign')}

        response = session.get(url,
                               background_callback=process_response,
                               headers=headers,
                               timeout=3,
                               proxies=proxy)

        # Add attributes to response
        response.market = market + str(index)
        response.url = request_input.get('url')
        response.headers = headers

        futures.append(response)

    for future in as_completed(futures):

        try:
            response_data = future.result().data

            if not response_data.get('success'):
                if response_data.get('message') == "INVALID_MARKET":
                    markets.remove(future.market)
                    logger.debug('Removed {}: invalid market ...'.format(future.market))
                continue

            response_dict[future.market] = response_data.get('result')
            if not response_dict[future.market]:
                if response_data.get('message') == "NO_API_RESPONSE":
                    raise ProxyError('NO API RESPONSE')

        except (ProxyError, ConnectTimeout, ConnectionError, ReadTimeout):

            api_retry = 0

            while True:

                if api_retry >= max_api_retry:
                    logger.debug('MAX API RETRY LIMIT ({}) REACHED. SKIPPING {}.'.format(str(max_api_retry),
                                                                                         future.market))
                    break

                r = randint(0, num_proxies - 1)
                proxy = configure_ip(proxies[r])

                try:
                    response = session.get(future.url,
                                           background_callback=process_response,
                                           headers=future.headers,
                                           timeout=2,
                                           proxies=proxy)
                    response_dict[future.market] = response.result().data.get('result')
                    if not response_dict[future.market]:
                        logger.debug('NO API RESPONSE, RETRYING: {} ...'.format(future.market))
                        api_retry += 1
                        continue

                    break

                except (ProxyError, ConnectTimeout, ConnectionError, ReadTimeout):
                    api_retry += 1
                    logger.debug('Retried API call failed for {}.'.format(future.market))

In [13]:
[response_dict.get(x)[0].get('TimeStamp') for x in response_dict]

['2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:51.193',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:51.193',
 '2017-12-09T06:23:36.287',
 '2017-12-09T06:23:36.287',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:27.397',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:41.943',
 '2017-12-09T06:25:27.397',
 '2017-12-09T06:23:33.94',
 '2017-12-09T06:25:51.193',
 '2017-12-09T06:25:51.193',
 '2017-12-09T06:25:51.193',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:51.193',
 '2017-12-09T06:25:28.647',
 '2017-12-09T06:25:41.943',
 '2017-12-09T06:25:21.49',
 '2017-12-09T06:25:28.647',
 '2017-12-09T06:25:28.647',
 '2017-12-09T06:25:41.943',
 '2017-12-09

In [20]:
[response_dict.get(x)[0].get('TimeStamp') for x in response_dict]

['2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:35.183',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:35.183',
 '2017-12-09T06:29:35.183',
 '2017-12-09T06:29:35.183',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:39.45',
 '2017-12-09T06:29:02.34',
 '2017-12-09T06:29:07.34',
 '2017-12-09T06:29:12.793',
 '2017-12-09T06:29:39.45',
 '2017-12-09T06:29:39.45',
 '2017-12-09T06:29:35.183',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:35.183',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:07.34',
 '2017-12-09T06:29:07.34',
 '2017-12-09T06:29:39.45',
 '2017-12-09T06:29:28.213',
 '2017-12-09T06:29:28.213',
